# QSM Reconstruction

In [ ]:
!run_2_qsm.py bids-new qsm-new --premade body --auto_yes --slurm a_barth general

In [ ]:
!for f in `find qsm-new3/qsm_final -name '*.nii*'`; do sub="${f/*sub/sub}"; sub="${sub/_ses*/}"; ses="${f/*_ses/ses}"; ses="${ses/_*/}"; cp "${f}" "bids-new2/${sub}/${ses}/extra_data/"; done

In [ ]:
!for f in `find qsm-new/ -wholename '*numpy_nibabel_twopass/mapflow/*/*echo-02*twopass.nii*'`; do sub="${f/*sub/sub}"; sub="${sub/_ses*/}"; ses="${f/*_ses/ses}"; ses="${ses/_*/}"; f2="${f/twopass1/twopass3}"; f2="${f2/echo-02/echo-04}"; ~/qsmxt/interfaces/nipype_interface_nonzeroaverage.py "${f}" "${f2}" "bids-new/${sub}-NEW/${ses}/extra_data/${sub}_${ses}_qsm_echo2-and-echo4.nii"; done

In [ ]:
import glob
import os
from qsmxt.interfaces import nipype_interface_nonzeroaverage as nza
subjects = ['sub-z0171603', 'sub-z0459570', 'sub-z1318033', 'sub-z1372775']
for subject in subjects:
    for session in os.listdir(glob.glob(os.path.join('qsm-new', 'workflow', f'{subject}*'))[0]):
        qsm_pattern = os.path.join('qsm-new', 'workflow', f'{subject}*', session, 'qsmxt*', 'numpy_nibabel_twopass', 'mapflow', '*', '*.nii*')
        qsm_files = sorted(glob.glob(qsm_pattern))
        qsm_files = [qsm_files[1], qsm_files[3]]
        nza.nonzero_average(qsm_files, mask_files=None, out_file=f'bids-new/{subject}-NEW/{session}/extra_data/{subject}_{session}_qsm_echo2-and-echo4.nii')

# Imports

In [1]:
import os
import glob
import nibabel as nib
import nilearn.image
import numpy as np
import enum
import json
from scipy.ndimage import label
import cv2
import scipy
from scipy.ndimage import binary_dilation, binary_erosion, generate_binary_structure, label

# Locate data

In [2]:
bids_dir = "bids-new"

In [3]:
session_dirs = []
for json_path in sorted(glob.glob(os.path.join(bids_dir, "sub*", "ses*", "anat", "*echo-01*mag*json"))):
    with open(json_path, 'r') as json_file:
        json_data = json.load(json_file)
        if json_data['ProtocolName'] in ["t2starME_qsm_tra_Iso1.4mm_INPHASE_bipolar_RUN_THIS_ONE"]:#, "wip_iSWI_fl3d_vibe", "wip_iSWI_fl3d_vibe_TRY THIS ONE"]:
            session_dirs.append(os.sep.join(os.path.split(json_path)[0].split(os.sep)[:-1]))
print(session_dirs)
for session_dir in session_dirs:
    subject = session_dir.split(os.sep)[1]
    


['bids-new/sub-z0034542/ses-20220715', 'bids-new/sub-z0171603-NEW/ses-20230731', 'bids-new/sub-z0182923/ses-20230705', 'bids-new/sub-z0186251/ses-20221107', 'bids-new/sub-z0237546/ses-20230508', 'bids-new/sub-z0317485/ses-20230517', 'bids-new/sub-z0445614/ses-20230510', 'bids-new/sub-z0705200/ses-20230104', 'bids-new/sub-z0755228/ses-20211108', 'bids-new/sub-z1167038/ses-20220315', 'bids-new/sub-z1181657/ses-20220315', 'bids-new/sub-z1262112/ses-20220314', 'bids-new/sub-z1372775-NEW/ses-20230308', 'bids-new/sub-z1472355/ses-20221222', 'bids-new/sub-z1568577/ses-20230510', 'bids-new/sub-z1571613/ses-20230519', 'bids-new/sub-z1728751/ses-20220328', 'bids-new/sub-z1778013/ses-20220715', 'bids-new/sub-z1818796/ses-20230313', 'bids-new/sub-z2007565/ses-20220715', 'bids-new/sub-z2900812/ses-20230523', 'bids-new/sub-z2904752/ses-20220826', 'bids-new/sub-z2952003/ses-20230614', 'bids-new/sub-z3171177/ses-20230313', 'bids-new/sub-z3278008/ses-20211109', 'bids-new/sub-z3393287/ses-20230524']


In [4]:
extra_files = sum((glob.glob(os.path.join(session_dir, "extra_data", "*.nii*")) for session_dir in session_dirs), [])

qsm_files2 = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*qsm.*")) for session_dir in session_dirs), []))
qsm_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*qsm_echo2-and-echo4.*")) for session_dir in session_dirs), []))
t2s_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*t2s*.nii*")) for session_dir in session_dirs), []))
r2s_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*r2s*.nii*")) for session_dir in session_dirs), []))
mag_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "magnitude_combined.nii")) for session_dir in session_dirs), []))
swi_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*swi.nii")) for session_dir in session_dirs), []))

gre_seg_raw_files = [extra_file for extra_file in extra_files if all(pattern in extra_file for pattern in ['_segmentation.', 'run'])]
gre_seg_clean_files = [seg_file.replace(".nii", "_clean.nii") for seg_file in gre_seg_raw_files if os.path.exists(seg_file.replace(".nii", "_clean.nii"))]

t1_files = [extra_file for extra_file in extra_files if any(pattern in extra_file for pattern in ['_T1w', '_t1_tra']) and not any(pattern in extra_file for pattern in ['_Pelvis_', '.json', '_resampled'])]
t1_files = [t1_file.replace("_resampled", "") for t1_file in t1_files]
t1_resampled_files = [t1_file.replace(".nii", "_resampled.nii") for t1_file in t1_files if os.path.exists(t1_file.replace(".nii", "_resampled.nii"))]

ct_files = [extra_file for extra_file in extra_files if 'resliced' in extra_file and any(pattern in extra_file for pattern in ['_na_', '_Pelvis_', '_NA']) and not any(pattern in extra_file for pattern in ['_t1_tra_', 'ATX', 'AXT', 'ROI', 'segmentation', '.json'])]
ct_seg_raw_files = sum((glob.glob(ct_file.replace(".nii", "_segmentation.nii")) for ct_file in ct_files), [])
ct_seg_clean_files = [ct_file.replace("_segmentation", "_segmentation_clean") for ct_file in ct_seg_raw_files if os.path.exists(ct_file.replace("_segmentation", "_segmentation_clean"))]

print(f"{len(ct_files)} CT images found.")
print(f"{len(ct_seg_raw_files)} raw CT segmentations found.")
print(f"{len(ct_seg_clean_files)} clean CT segmentations found.")
print(f"{len(qsm_files)} QSM images found.")
print(f"{len(qsm_files2)} QSM (2) images found.")
print(f"{len(mag_files)} magnitude images found.")
print(f"{len(t2s_files)} T2* maps found.")
print(f"{len(r2s_files)} R2* maps found.")
print(f"{len(swi_files)} SWI maps found.")
print(f"{len(t1_files)} T1w files found.")
print(f"{len(t1_resampled_files)} resampled T1w files found.")
print(f"{len(gre_seg_raw_files)} raw GRE segmentations found.")
print(f"{len(gre_seg_clean_files)} clean GRE segmentations found.")

26 CT images found.
26 raw CT segmentations found.
26 clean CT segmentations found.
26 QSM images found.
17 QSM (2) images found.
26 magnitude images found.
26 T2* maps found.
26 R2* maps found.
26 SWI maps found.
26 T1w files found.
26 resampled T1w files found.
26 raw GRE segmentations found.
26 clean GRE segmentations found.


# Clean up segmentations

In [19]:
class SegType(enum.Enum):
    NO_LABEL = 0
    PROSTATE = 1
    GOLD_SEED = 2
    CALCIFICATION = 3

for i in range(len(qsm_files)):
    # load files
    qsm_nii = nib.load(qsm_files[i])
    seg_nii = nib.load(gre_seg_raw_files[i])

    # get image data
    qsm = qsm_nii.get_fdata()
    seg = np.array(seg_nii.get_fdata(), dtype=np.uint8)

    # get prostate values
    prostate_values = qsm[seg == SegType.PROSTATE.value]

    stdevs = 2.5

    # automated gold seed mask
    seed_mask_labelled, num_labels = label(
        input=seg == SegType.GOLD_SEED.value,
        structure=np.ones((3, 3, 3))
    )
    seed_mask = np.zeros(seg.shape, dtype=np.uint8)
    for seed_id in range(1, num_labels + 1):
        seed_i_mask = seed_mask_labelled == seed_id
        seed_i_mean = qsm[seed_i_mask].mean()
        if seed_i_mean > prostate_values.mean() + 1*prostate_values.std():
            seed_i_mask = np.logical_and(seed_i_mask, qsm > prostate_values.mean() + stdevs*prostate_values.std())
            seed_i_mask = np.logical_and(binary_dilation(seed_i_mask), (qsm > np.mean(prostate_values) + stdevs*np.std(prostate_values)))
        else:
            seed_i_mask = np.logical_and(seed_i_mask, qsm < prostate_values.mean() - stdevs*prostate_values.std())
            seed_i_mask = np.logical_and(binary_dilation(seed_i_mask), (qsm < np.mean(prostate_values) - stdevs*np.std(prostate_values)))
        seed_mask = np.logical_or(seed_mask, seed_i_mask)
    structure = generate_binary_structure(3, 1) # 3 for 3D image, 2 for full connectivity
    labeled_mask, num_features = label(seed_mask, structure)
    for idx in range(1, num_features + 1):
        if np.sum(labeled_mask == idx) == 1:
            seed_mask[labeled_mask == idx] = 0
    seed_mask = np.logical_and(seed_mask, seg == SegType.GOLD_SEED.value)

    # automated calcification mask
    calc_mask = np.array(seg == SegType.CALCIFICATION.value, dtype=np.uint8)
    calc_mask = np.logical_and(calc_mask, qsm < prostate_values.mean() - stdevs*prostate_values.std())
    calc_mask = np.logical_and(binary_dilation(calc_mask), (qsm < np.mean(prostate_values) - stdevs*np.std(prostate_values)))
    structure = generate_binary_structure(3, 1) # 3 for 3D image, 2 for full connectivity
    labeled_mask, num_features = label(calc_mask, structure)
    for idx in range(1, num_features + 1):
        if np.sum(labeled_mask == idx) == 1:
            calc_mask[labeled_mask == idx] = 0
    calc_mask = np.logical_and(calc_mask, seg == SegType.CALCIFICATION.value)

    # setup final values
    seg[seg == SegType.GOLD_SEED.value] = 0
    seg[seg == SegType.CALCIFICATION.value] = 0
    seg[seg == SegType.PROSTATE.value] = 0

    seg[seed_mask] = 1
    seg[calc_mask] = 2
    
    # save result using original file extension
    filename = f"{gre_seg_raw_files[i].replace('.nii', '_clean.nii')}"
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename=filename)
    print(f"Saved {filename}")

Saved bids-new/sub-z0034542/ses-20220715/extra_data/sub-z0034542_ses-20220715_run-01_segmentation_clean.nii.gz
Saved bids-new/sub-z0171603-NEW/ses-20230731/extra_data/sub-z0171603_ses-20230731_run-01_segmentation_clean.nii
Saved bids-new/sub-z0182923/ses-20230705/extra_data/sub-z0182923_ses-20230705_run-01_segmentation_clean.nii
Saved bids-new/sub-z0186251/ses-20221107/extra_data/sub-z0186251_ses-20221107_run-01_segmentation_clean.nii.gz
Saved bids-new/sub-z0237546/ses-20230508/extra_data/sub-z0237546_ses-20230508_run-01_segmentation_clean.nii
Saved bids-new/sub-z0317485/ses-20230517/extra_data/sub-z0317485_ses-20230517_run-01_segmentation_clean.nii
Saved bids-new/sub-z0445614/ses-20230510/extra_data/sub-z0445614_ses-20230510_run-01_segmentation_clean.nii
Saved bids-new/sub-z0705200/ses-20230104/extra_data/sub-z0705200_ses-20230104_run-01_segmentation_clean.nii
Saved bids-new/sub-z0755228/ses-20211108/extra_data/sub-z0755228_ses-20211108_run-01_segmentation_clean.nii.gz
Saved bids-new/

In [20]:
class SegType(enum.Enum):
    NO_LABEL = 0
    PROSTATE = 1
    GOLD_SEED = 2
    CALCIFICATION = 3

for i in range(len(qsm_files)):
    # load files
    ct_nii = nib.load(ct_files[i])
    seg_nii = nib.load(ct_seg_raw_files[i])

    # get image data
    ct = ct_nii.get_fdata()
    seg = np.array(seg_nii.get_fdata(), dtype=np.uint8)

    # get prostate values
    prostate_values = ct[seg == SegType.PROSTATE.value]

    # seed mask
    seed_mask = np.logical_and(seg == SegType.GOLD_SEED.value, ct > prostate_values.mean() + 4*prostate_values.std())
    seed_mask = np.logical_and((ct > np.mean(prostate_values) + 4*np.std(prostate_values)), binary_dilation(seed_mask))
    structure = generate_binary_structure(3, 1)
    labeled_mask, num_features = label(seed_mask, structure)
    for idx in range(1, num_features + 1):
        if np.sum(labeled_mask == idx) == 1:
            seed_mask[labeled_mask == idx] = 0
    seed_mask = np.logical_and(seed_mask, seg == SegType.GOLD_SEED.value)

    # calcification mask
    calc_mask = np.logical_and(seg != SegType.GOLD_SEED, ct > np.mean(prostate_values) + 4*np.std(prostate_values))
    calc_mask = np.logical_and((ct > np.mean(prostate_values) + 4*np.std(prostate_values)), binary_dilation(calc_mask))
    calc_mask = np.logical_and((ct > np.mean(prostate_values) + 4*np.std(prostate_values)), binary_dilation(calc_mask))
    structure = generate_binary_structure(3, 1)
    labeled_mask, num_features = label(calc_mask, structure)
    for idx in range(1, num_features + 1):
        if np.sum(labeled_mask == idx) == 1:
            calc_mask[labeled_mask == idx] = 0
    calc_mask = np.logical_and(calc_mask, seg == SegType.CALCIFICATION.value)
    
    # setup final values
    seg[seg == SegType.GOLD_SEED.value] = 0
    seg[seg == SegType.CALCIFICATION.value] = 0
    seg[seg == SegType.PROSTATE.value] = 0

    seg[seed_mask] = 1
    seg[calc_mask] = 2
    
    # save result using original file extension
    filename = f"{ct_seg_raw_files[i].replace('.nii', '_clean.nii')}"
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename=filename)
    print(f"Saved {filename}")

Saved bids-new/sub-z0034542/ses-20220715/extra_data/5_na_20220715092125_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z0171603-NEW/ses-20230731/extra_data/6_NA_20230731112304_6_resliced_segmentation_clean.nii
Saved bids-new/sub-z0182923/ses-20230705/extra_data/5_NA_20230705093737_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z0186251/ses-20221107/extra_data/5_na_20221107081460_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z0237546/ses-20230508/extra_data/7_na_20230508102530_7_resliced_segmentation_clean.nii
Saved bids-new/sub-z0317485/ses-20230517/extra_data/5_NA_20230517091908_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z0445614/ses-20230510/extra_data/5_16_pelvis_16_Pelvis_20230510090050_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z0705200/ses-20230104/extra_data/5_11_pelvis_11_Pelvis_20230104113938_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z0755228/ses-20211108/extra_data/Z0755228_20211109_Pelvis_2.0_HD_FoV_iMAR_20211109085844_5_resl

# Combined magnitude

In [ ]:
for session_dir in session_dirs:
    mag_images = sorted(glob.glob(os.path.join(session_dir, "anat", "*part-mag*nii*")))
    mag_images_even = [mag_images[i] for i in range(1, len(mag_images), 2)]
    nii = nib.load(mag_images_even[0])
    mag_4d = np.array([nib.load(mag_images_even[i]).get_fdata() for i in range(len(mag_images_even))])
    mag_combined = np.sqrt(np.sum(np.square(mag_4d), axis=0))
    nib.save(nib.Nifti1Image(mag_combined, header=nii.header, affine=nii.affine), os.path.join(session_dir, "extra_data", "magnitude_combined.nii"))

# Combine QSM SNR-weighted

In [7]:

for session_dir in session_dirs:
    subject_id = session_dir.split(os.sep)[1]
    session_id = session_dir.split(os.sep)[2]
    print(subject_id, session_id)
    qsm_pattern = os.path.join('qsm', 'workflow', f'{subject_id}*', session_id, 'qsmxt*', 'numpy_nibabel_twopass', 'mapflow', '*', '*.nii*')
    qsm_files = sorted(glob.glob(qsm_pattern))
    qsm_files = [qsm_files[i] for i in range(len(qsm_files)) if i % 2 == 1]
    mag_pattern = os.path.join('bids-new', subject_id, session_id, 'anat', '*part-mag*nii*')
    mag_files = sorted(glob.glob(mag_pattern))
    mag_files = [mag_files[i] for i in range(len(mag_files)) if i % 2 == 1]

    qsm_images = [nib.load(f) for f in qsm_files]
    magnitude_images = [nib.load(f) for f in mag_files]

    # 2. Weights 1
    epsilon = 1e-3  # Small constant to avoid division by very small numbers
    weights = [1 / (img.get_fdata() + epsilon) for img in magnitude_images]
    sum_weights = np.sum(np.stack(weights, axis=-1), axis=-1)
    normalized_weights = [w / sum_weights for w in weights]

    # 2. Weights 2
    #weights = [1 / np.mean(img.get_fdata()) for img in magnitude_images]
    #normalized_weights = weights / np.sum(weights)
    #weighted_qsm_data = np.zeros_like(qsm_images[0].get_fdata())

    # 3. Calculate the weighted average of the QSMs:
    weighted_qsm_data = np.zeros_like(qsm_images[0].get_fdata())
    for i, qsm_img in enumerate(qsm_images):
        weighted_qsm_data += normalized_weights[i] * qsm_img.get_fdata()

    # Create a new NIfTI image with the weighted QSM data
    weighted_qsm_img = nib.Nifti1Image(weighted_qsm_data, qsm_images[0].affine, qsm_images[0].header)

    # 4. Save the resulting weighted QSM:
    nib.save(weighted_qsm_img, f"{subject_id}_{session_id}_weighted_qsm.nii.gz")
    print("1")

sub-z0034542 ses-20220715
1
sub-z0171603-NEW ses-20230731
1
sub-z0182923 ses-20230705
1
sub-z0186251 ses-20221107
1
sub-z0237546 ses-20230508
1
sub-z0317485 ses-20230517
1
sub-z0445614 ses-20230510
1
sub-z0705200 ses-20230104
1
sub-z0755228 ses-20211108
1
sub-z1167038 ses-20220315
1
sub-z1181657 ses-20220315
1
sub-z1262112 ses-20220314
1
sub-z1372775-NEW ses-20230308
1
sub-z1472355 ses-20221222
1
sub-z1568577 ses-20230510
1
sub-z1571613 ses-20230519
1
sub-z1728751 ses-20220328
1
sub-z1778013 ses-20220715
1
sub-z1818796 ses-20230313
1
sub-z2007565 ses-20220715
1
sub-z2900812 ses-20230523
1
sub-z2904752 ses-20220826
1
sub-z2952003 ses-20230614
1
sub-z3171177 ses-20230313
1
sub-z3278008 ses-20211109
1
sub-z3393287 ses-20230524
1


# Reslice T1w images

In [ ]:
for i in range(len(t1_files)):
    print(t1_files[i])
    t1_nii = nib.load(t1_files[i])
    mag_nii = nib.load(mag_files[i])
    t1_resampled = nilearn.image.resample_img(t1_nii, target_affine=mag_nii.affine, target_shape=mag_nii.header.get_data_shape())
    filename = t1_files[i].replace(".nii", "_resampled.nii")
    nib.save(t1_resampled, filename)

# T2* Maps

In [ ]:
!cat {session_dirs[0]}/anat/*phase*json | grep -i echotime\"

In [ ]:
for session_dir in session_dirs:
    print(session_dir)
    !./make_t2starmaps.jl --magnitude {os.path.join(session_dir, "anat", "*mag*.nii*")} --TEs "[0.00246, 0,00492, 0.00738, 0,00984, 0.0123, 0.0148, 0.01722]" --t2starmap {os.path.join(session_dir, "extra_data", "t2starmap.nii")} --r2starmap {os.path.join(session_dir, "extra_data", "r2starmap.nii")}

# SWI

In [ ]:
!julia -e "import Pkg; Pkg.add(Pkg.PackageSpec(url=\"https://github.com/korbinian90/CLEARSWI.jl\"))"

In [ ]:
#./make_swis.jl --phase bids-new/sub-z0034542/ses-20220715/anat/sub-z0034542_ses-20220715_run-01_echo-*_part-phase_MEGRE.nii --magnitude bids-new/sub-z0034542/ses-20220715/anat/sub-z0034542_ses-20220715_run-01_echo-*_part-mag_MEGRE.nii --TEs [0.00492,0.00738] --swi-out swi-out.nii --mip-out mip-out.nii

In [ ]:
for session_dir in session_dirs:
    !echo {session_dir}
    !./make_swis.jl \
        --magnitude {os.path.join(session_dir, "anat", "*mag*.nii*")} \
        --phase {os.path.join(session_dir, "anat", "*phase*.nii*")} \
        --TEs "[0.00246, 0.00492, 0.00738, 0.00984, 0.0123, 0.0148, 0.01722]" \
        --swi-out {os.path.join(session_dir, "extra_data", "swi.nii")}